In [3]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

alpha = 1
threshold = 60

adaptive_background = None

virtual_scene = cv2.imread('virtual_scene.jpg')
virtual_scene = cv2.resize(virtual_scene, (640, 480))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

    # Adaptive Background Image
    if adaptive_background is None:
        adaptive_background = frame.astype(float)
    else:
        adaptive_background = alpha * adaptive_background + (1 - alpha) * frame # B(t)=alphaB(t-1)+(1-alpha)I(t)

    # Adaptive Background Subtraction
    # S(t)=abs(I(t)-B(t))
    foreground_mask = cv2.absdiff(frame, adaptive_background.astype(np.uint8))
    
    # Foreground Detection
    foreground_detection = np.where(foreground_mask > threshold, frame, 0) # F(t)=I(t), if S(t) > Threshold

    # Synthesis an image by keeping the foreground, and replacing the background using any virtual scene image of your choice.
    virtual_scene_resized = cv2.resize(virtual_scene, (foreground_detection.shape[1], foreground_detection.shape[0]))
    synthesized_image = np.where(foreground_detection > 0, foreground_detection, virtual_scene_resized)

    cv2.imshow('123', adaptive_background)
    cv2.imshow('Captured Image', frame)
    cv2.imshow('Adaptive Background Subtraction', foreground_mask)
    cv2.imshow('Foreground Detection', foreground_detection)
    cv2.imshow('Synthesized Image', synthesized_image)
 
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()